In [1]:
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import time
import struct
import hashlib

from utils import *
from dut import *
from rv_utils import *

%matplotlib inline

In [29]:
BIGVOLT = 3.281
def BV(x):
    return round(BIGVOLT * x / 65536, 3)

In [2]:
dut = DUT("/dev/tty.usbmodem101", 115200)

In [ ]:
dut.write_flash(0, 4096, prep_flash(fw))

In [ ]:
dut.write_flash(0, 4096, prep_flash(INF_LOOP))

In [26]:
def run_prog(prog, clkdiv, caravel_voltage, proj_voltage, ms=200, reprogram=True, verbose=True):
    if reprogram:
        fw = assemble_file(prog)
        dat = prep_flash(fw)
        with CriticalSection():
            dut.write_flash(0, len(dat), dat)
    
    with CriticalSection():
        dut.set_clkdiv(clkdiv)
        dut.set_project_voltage(proj_voltage)
        dut.set_caravel_voltage(caravel_voltage)

        dut.set_reset(True)
        dut.reset_dc()
        dut.reset_ctr()
        dut.enable_for_millis(ms)
        
        out = bytes(dut.get_dc_buf())
        
        if verbose:
            print(BV(caravel_voltage), BV(proj_voltage), dut.get_ctr())
            print(dut.get_dc_len(), out)
        
    return out

In [4]:
def padding(msg):
    lo = 8 * len(msg)
    msg = msg + b"\x80"
    while (len(msg) + 8) % 64 != 0:
        msg = msg + b"\x00"
    msg = msg + struct.pack(">Q", lo)
    return msg

def compute_sha(string, clkdiv=12, caravel_voltage=22500, proj_voltage=19500, ms=800, reprogram=True, verbose=True):
    
    with open("../caravel_fw/sha_template.S", "r") as f:
        dat = f.read()
            
    # Generate the assembly to load the string
    string = padding(string.encode())
    string_words = [string[i:i+4].hex() for i in range(0, len(string), 4)]
    load_asm = "".join([f"li t1, 0x{word}\nsw t1, 4(t0)\n" for word in string_words])

    dat = dat.replace(r"{{TEMPLATE}}", load_asm)
    
    with open("/tmp/shaprog.S", "w+") as f:
        f.write(dat)
    
    res = run_prog("/tmp/shaprog.S", clkdiv, caravel_voltage, proj_voltage, ms=ms, reprogram=reprogram, verbose=verbose)
    return res.hex()

In [ ]:
s = compute_sha("hello", 24, 21500, 18500, ms=800, reprogram=False).strip()

In [ ]:
hashlib.sha256(b"hello").hexdigest()

In [ ]:
s == hashlib.sha256(b"hello").hexdigest()

In [ ]:
compute_sha("hello", 24, 21500, vvv, ms=800, reprogram=True)

for vvv in range(17000, 19000, 100):
    s = compute_sha("hello", 24, 21500, vvv, ms=800, reprogram=False, verbose=False).strip()
    print("RES", vvv, s == hashlib.sha256(b"hello").hexdigest())

In [23]:
_ = run_prog("../caravel_fw/sha_ident.S", 12, 22500, 19000, ms=800, reprogram=True)


22500 19000 4173
64 b'SHA256 Core by Anish Singhani (12/15/2020 18:23 PST)   \x00\x00\x00\x00\x00\x00\x00\x00\x00'
22500 19000 4173
64 b'AES128/256 Core by Anish Singhani (12/15/2020 18:23 PST)   \x00\x00\x00\x00\x00'


In [43]:

for i in range(30000, 35000, 200):
    _ = run_prog("../caravel_fw/sha_ident.S", 12, 20000, i, ms=200, reprogram=False)
    print(BV(i), _ == b"SHA256 Core by Ani")

1.001 1.502 377
18 b'SHA256 Core by Ani'
1.502 True
1.001 1.512 377
18 b'SHA256 Core by Ani'
1.512 True
1.001 1.522 377
18 b'SHA256 Core by Ani'
1.522 True
1.001 1.532 377
18 b'SHA256 Core by Ani'
1.532 True
1.001 1.542 377
18 b'SHA256 Core by Ani'
1.542 True
1.001 1.552 377
18 b'SHA256 Core by Ani'
1.552 True
1.001 1.562 377
18 b'SHA256 Core by Ani'
1.562 True
1.001 1.572 377
18 b'SHA256 Core by Ani'
1.572 True
1.001 1.582 377
18 b'SHA256 Core by Ani'
1.582 True
1.001 1.592 377
18 b'SHA256 Core by Ani'
1.592 True
1.001 1.602 377
18 b'SHA256 Core by Ani'
1.602 True
1.001 1.612 377
18 b'SHA256 Core by Ani'
1.612 True
1.001 1.622 377
18 b'SHA256 Core by Ani'
1.622 True
1.001 1.632 377
18 b'SHA256 Core by Ani'
1.632 True
1.001 1.642 377
18 b'SHA256 Core by Ani'
1.642 True
1.001 1.652 377
18 b'SHA256 Core by Ani'
1.652 True
1.001 1.662 377
18 b'SHA256 Core by Ani'
1.662 True
1.001 1.672 377
18 b'SHA256 Core by Ani'
1.672 True
1.001 1.682 377
18 b'SHA256 Core by Ani'
1.682 True
1.001 1.692 

In [48]:
for i in range(18000, 35000, 200):
    print(run_prog("../caravel_fw/sha_test_long.S", 12, 21000, i, ms=800, reprogram=False).hex() == "41812dc60561798dc0cc6e574b641893ecf4186d4097283af4a6cff3eddaa4a0")
    

    

1.051 0.901 21
0 b''
False
1.051 0.911 10126
32 b"\xc6\xe7\x00\xc6O=36\xf2\x0f\xe0\x04\xad\xc7@\xe4'\x8e?\xf1~\x87U\x13z\xe0\x02\x92s\xc3\x05\x13"
False
1.051 0.921 10126
32 b'\x97J\x03\xdex\x13\xd7\x16\xd0CF\x00\xc6\x1c\x11p\xf0\x8c\xd7l\x0f\xe4\xc3\x91At\x1b\xeb\x0e\xd9c\xbc'
False
1.051 0.931 10126
32 b'\xd0py|\x95x\xfa\x83\x07-\xd3y\xcc\x18\x03\x9c-\xd5\x94\xf4\xa9\xd7wR\xf1\xad\xd0\x98:\x10\xc1\xea'
False
1.051 0.941 10126
32 b'\xce\x1f\xf9l\x8a\xd3\x9e\xad\x1e\xb4\x89{\x1f\x9e<\xff\xf1ts\xbd\xdc\x1d\xeee\xd5\x8fQ\xe0}\x97\xbf\xd2'
False
1.051 0.951 10126
32 b'A\x81-\xc6\x05ay\x8d\xc0\xccnWKd\x18\x93\xec\xf4\x18m@\x97(:\xf4\xa6\xcf\xf3\xed\xda\xa4\xa0'
True
1.051 0.961 10126
32 b'A\x81-\xc6\x05ay\x8d\xc0\xccnWKd\x18\x93\xec\xf4\x18m@\x97(:\xf4\xa6\xcf\xf3\xed\xda\xa4\xa0'
True
1.051 0.971 10126
32 b'A\x81-\xc6\x05ay\x8d\xc0\xccnWKd\x18\x93\xec\xf4\x18m@\x97(:\xf4\xa6\xcf\xf3\xed\xda\xa4\xa0'
True
1.051 0.981 10126
32 b'A\x81-\xc6\x05ay\x8d\xc0\xccnWKd\x18\x93\xec\xf4\x18m@\x97(:\x

1.051 1.582 10126
32 b'A\x81-\xc6\x05ay\x8d\xc0\xccnWKd\x18\x93\xec\xf4\x18m@\x97(:\xf4\xa6\xcf\xf3\xed\xda\xa4\xa0'
True
1.051 1.592 10126
32 b'A\x81-\xc6\x05ay\x8d\xc0\xccnWKd\x18\x93\xec\xf4\x18m@\x97(:\xf4\xa6\xcf\xf3\xed\xda\xa4\xa0'
True
1.051 1.602 10126
32 b'A\x81-\xc6\x05ay\x8d\xc0\xccnWKd\x18\x93\xec\xf4\x18m@\x97(:\xf4\xa6\xcf\xf3\xed\xda\xa4\xa0'
True
1.051 1.612 10126
32 b'A\x81-\xc6\x05ay\x8d\xc0\xccnWKd\x18\x93\xec\xf4\x18m@\x97(:\xf4\xa6\xcf\xf3\xed\xda\xa4\xa0'
True
1.051 1.622 10126
32 b'A\x81-\xc6\x05ay\x8d\xc0\xccnWKd\x18\x93\xec\xf4\x18m@\x97(:\xf4\xa6\xcf\xf3\xed\xda\xa4\xa0'
True
1.051 1.632 10126
32 b'A\x81-\xc6\x05ay\x8d\xc0\xccnWKd\x18\x93\xec\xf4\x18m@\x97(:\xf4\xa6\xcf\xf3\xed\xda\xa4\xa0'
True
1.051 1.642 10126
32 b'A\x81-\xc6\x05ay\x8d\xc0\xccnWKd\x18\x93\xec\xf4\x18m@\x97(:\xf4\xa6\xcf\xf3\xed\xda\xa4\xa0'
True
1.051 1.652 10126
32 b'A\x81-\xc6\x05ay\x8d\xc0\xccnWKd\x18\x93\xec\xf4\x18m@\x97(:\xf4\xa6\xcf\xf3\xed\xda\xa4\xa0'
True
1.051 1.662 10126
32 b'A

#### 

In [27]:
def scan_voltage(vmin, vmax, vstep, clkdiv=None, verbose=True, enable_ms=20):
    if clkdiv is not None: dut.set_clkdiv(clkdiv)
    dut.set_project_voltage(21050)
    
    result = {}

    for voltage in range(vmin, vmax, vstep):
        with CriticalSection():
            dut.set_caravel_voltage(voltage)

            dut.set_reset(True)
            dut.reset_ctr()
            dut.enable_for_millis(enable_ms)
            if verbose:
                print(BV(voltage), dut.get_ctr())
                
            result[voltage] = dut.get_ctr()
            
    return result

def scan_repeatability(vmin, vmax, vstep, expmin, expmax, clkdiv=None, verbose=True, repeats=20, enable_ms=20):
    if clkdiv is not None: dut.set_clkdiv(clkdiv)
    dut.set_project_voltage(21050)
    
    result = {}

    for voltage in range(vmin, vmax, vstep):
        success = 0
        for i in range(repeats):
            with CriticalSection():
                dut.set_caravel_voltage(voltage)

                dut.set_reset(True)
                dut.reset_ctr()
                dut.enable_for_millis(enable_ms)
                
                if expmin < dut.get_ctr() and dut.get_ctr() < expmax:
                    success += 1
                
        if verbose:
            print(BV(voltage), round(success / repeats, 2))

        result[voltage] = round(success / repeats, 2)
            
    return result

In [33]:
o = scan_voltage(19000, 30000, 100, clkdiv=12)

0.951 5
0.956 35
0.961 37
0.966 37
0.971 37
0.976 37
0.981 37
0.986 37
0.991 37
0.996 37
1.001 37
1.006 37
1.011 37
1.016 37
1.021 37
1.026 37
1.031 37
1.036 37
1.041 37
1.046 37
1.051 37
1.056 37
1.061 37
1.066 37
1.071 37
1.076 37
1.081 37
1.086 37
1.091 37
1.096 37
1.101 37
1.106 37
1.111 37
1.116 37
1.121 37
1.126 37
1.131 37
1.136 37
1.141 37
1.146 37
1.151 37
1.156 37
1.161 37
1.166 37
1.171 37
1.177 37
1.182 37
1.187 37
1.192 37
1.197 37
1.202 37
1.207 37
1.212 37
1.217 37
1.222 37
1.227 37
1.232 37
1.237 37
1.242 37
1.247 37
1.252 37
1.257 11
1.262 5
1.267 4
1.272 4
1.277 4
1.282 4
1.287 4
1.292 4
1.297 4
1.302 4
1.307 4
1.312 4
1.317 4
1.322 4
1.327 4
1.332 4
1.337 3
1.342 3
delayed SIGINT
1.347 3


KeyboardInterrupt: 

In [32]:
o = scan_repeatability(18000, 25000, 100, 15, 21, repeats=20, clkdiv=12)

0.901 0.0
0.906 0.0
0.911 0.0
0.916 0.0
delayed SIGINT


KeyboardInterrupt: 